# Игра "камень, ножницы, бумага"

Установка необходимых библиотек

In [ ]:
%pip install -q -U pandas

In [ ]:
%pip install -q -U matplotlib

In [ ]:
%pip install -q -U seaborn

In [ ]:
%pip install -q -U kaggle_environments

Импорт библиотек и функций

In [1]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate


Ходы игроков представлены целочисленными константами:
+ 0 - "камень"
+ 1 - "бумага"
+ 2 - "ножницы"

Функция, возвращающая ход, который нужно сделать, чтобы выиграть оппонента, на основании его предполагаемого хода *opponent_action*

In [2]:
def action_to_win(opponent_action):
    if opponent_action == 0:
        return 1
    elif opponent_action == 1:
        return 2
    elif opponent_action == 2:
        return 0
    else:
        return -1


Атрибут *observation.lastOpponentAction* присутствует не всегда, поэтому в функциях агентов делается проверка его существования вызовом *hasattr(obj, name)*

Примеры игровых агентов

In [3]:
# агент, выдающий всегда "камень"
def rock_agent(observation, configuration):
    return 0


# агент, выдающий всегда "бумага"
def paper_agent(observation, configuration):
    return 1


# агент, выдающий всегда "ножницы"
def scissors_agent(observation, configuration):
    return 2


# агент, копирующий предыдущий ход оппонента
def copy_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)


# агент, предполагающий, что оппонент меняет свой ход на каждом шаге
def reactionary_agent(observation, configuration):
    if observation.step > 0:
        expected_opponent_actions = [0, 1, 2]
        expected_opponent_actions.remove(observation.lastOpponentAction)
        expected_opponent_action = expected_opponent_actions[random.randrange(0, 2)]
        return action_to_win(expected_opponent_action)
    else:
        return random.randrange(0, configuration.signs)


# агент, играющий поочерёдно "камень", "бумага", "ножницы"
seq_opponent_agent_action = None
def sequental_agent(observation, configuration):
    global seq_opponent_agent_action
    if observation.step == 0:
        seq_opponent_agent_action = random.randrange(0, configuration.signs)
    seq_opponent_agent_action = (seq_opponent_agent_action + 1) % configuration.signs
    return seq_opponent_agent_action


# агент, выдающий на каждом шаге случайный ход
def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)


# агент, выбирающий случайно или "камень", или "бумага"
def rock_paper_agent(observation, configuration):
    possible_action = random.randrange(0, 2)
    return 0 if possible_action == 0 else 1


# агент, выбирающий случайно или "камень", или "ножницы"
def rock_scissors_agent(observation, configuration):
    possible_action = random.randrange(0, 2)
    return 0 if possible_action == 0 else 2


# агент, выбирающий случайно или "бумага", или "ножницы"
def paper_scissors_agent(observation, configuration):
    possible_action = random.randrange(0, 2)
    return 1 if possible_action == 0 else 2


# агент, копящий статистику всех ходов оппонента, определяющий его самый частый ход
# и выбирающий ход, чтобы победить с наибольшей вероятностью
stat_opp_action_counts = None
def statistical_agent(observation, configuration):
    global stat_opp_action_counts
    if observation.step == 0:
        stat_opp_action_counts = [0, 0, 0]
    if hasattr(observation, 'lastOpponentAction') and \
    observation.lastOpponentAction in list(range(0, configuration.signs)):
        stat_opp_action_counts[observation.lastOpponentAction] += 1
    frequent_opponent_action = stat_opp_action_counts.index(max(stat_opp_action_counts))
    return action_to_win(frequent_opponent_action)


# агент, копящий статистику последних 10 ходов оппонента, определяющий его самый частый ход
# за эти 10 ходов и выбирающий ход, чтобы победить с наибольшей вероятностью
react_stat_opp_actions = None
def react_statistical_agent(observation, configuration):
    global react_stat_opp_actions
    if observation.step == 0:
        react_stat_opp_actions = []
    if hasattr(observation, 'lastOpponentAction'):
        react_stat_opp_actions.append(observation.lastOpponentAction)
    if (len(react_stat_opp_actions) > 10):
        react_stat_opp_actions = react_stat_opp_actions[1:]
    opponent_action_counts = [react_stat_opp_actions.count(action) for action in range(0, configuration.signs)]
    frequent_opponent_action = opponent_action_counts.index(max(opponent_action_counts))
    return action_to_win(frequent_opponent_action)


Пример игры между агентами

In [4]:
evaluate(
    environment="rps", #выбор игровой среды "камень, ножницы, бумага"
    configuration={"episodeSteps": 100}, #количество ходов игровых агентов
    agents=[rock_agent, scissors_agent], #игровые агенты
    num_episodes=1 #число эпизодов
)


[[99.0, -99.0]]

Прогон игр между различными агентами и вывод турнирной таблицы по всем агентам.

In [24]:
env = "rps" #выбор игровой среды "камень, ножницы, бумага"
conf = {"episodeSteps": 100} #количество ходов игровых агентов
n_episodes = 1 #число эпизодов

#все игровые агенты и их очки
agents = {
          'rock': [rock_agent, 0, 0, 0],
          'paper': [paper_agent, 0, 0, 0],
          'scissors': [scissors_agent, 0, 0, 0],
          'copy': [copy_agent, 0, 0, 0],
          'reactionary' : [reactionary_agent, 0, 0, 0],
          'sequental' : [sequental_agent, 0, 0, 0],
          'random' : [random_agent, 0, 0, 0],
          'rock_paper' : [rock_paper_agent, 0, 0, 0],
          'rock_scissors' : [rock_scissors_agent, 0, 0, 0],
          'paper_scissors' : [paper_scissors_agent, 0, 0, 0],
          'statistical' : [statistical_agent, 0, 0, 0],
          'react_statistical' : [react_statistical_agent, 0, 0, 0]
          }

for agent_name, agent_inst in agents.items():
    for opponent_name, opponent_inst in agents.items():        
        score = evaluate(environment=env, configuration=conf,
            agents=[agent_inst[0], opponent_inst[0]], num_episodes=n_episodes)[0]
        # победил сам агент
        if score[0] > score[1]:
            agent_inst[1] += 1
            opponent_inst[2] += 1
        # победил оппонент агента
        elif score[0] < score[1]:
            opponent_inst[1] += 1
            agent_inst[2] += 1
        # ничья
        else:
            agent_inst[3] += 1
            opponent_inst[3] += 1

# сортировка агентов по числу побед по убыванию
agents_list = sorted(agents.items(), key=lambda item: item[1][1], reverse=True)

# вывод турнирной таблицы
print(f"| {'AGENT':<20} | {'WINS':>7} | {'LOSSES':>7} | {'DRAWS':>7} |")
for agent_name, agent_inst in agents_list:
    print(f"| {agent_name:<20} | {agent_inst[1]:>7} | {agent_inst[2]:>7} | {agent_inst[3]:>7} |")


| AGENT                |    WINS |  LOSSES |   DRAWS |
| react_statistical    |      16 |       0 |       8 |
| statistical          |      14 |       4 |       6 |
| rock                 |       6 |       8 |      10 |
| paper                |       6 |       8 |      10 |
| scissors             |       6 |       8 |      10 |
| rock_paper           |       5 |       8 |      11 |
| sequental            |       4 |       1 |      19 |
| rock_scissors        |       4 |       8 |      12 |
| paper_scissors       |       4 |       7 |      13 |
| copy                 |       1 |       3 |      20 |
| reactionary          |       0 |      11 |      13 |
| random               |       0 |       0 |      24 |


По итогам прогонов игр между разными агентами наилучшие результаты показывают агенты *react_statistical_agent* и *statistical_agent*, наихудшие результаты - агенты *copy_agent*, *reactionary_agent* и *random_agent*.